In [ ]:
import numpy as np
import sys

class LBM_2_Carlemann:
    def __init__(self):
        # Constants for the D1Q3 lattice (1D, 3 velocities)
        self.w = np.array([2/3, 1/6, 1/6])  # Weights for D1Q3 lattice
        self.e = np.array([0, 1, -1])  # Lattice directions: [0, +1, -1]
        self.c_s = 1 / np.sqrt(3)  # Speed of sound for D1Q3 lattice...is this the delta_x\delta_t ??
        self.g = 9.81  # Acceleration due to gravity (m/s^2)
        self.kn = 0.1 #Knudsen number, much less than 1 for chapman-enskogg expansion

        # Parameters
        self.tau = 1.0  # Relaxation time
        self.Nx = 3  # Number of grid points...my code for the F matrices makes the kernel die if this number is 81 or higher
        #self.L = 1.0  # Length of the domain (in meters)

        # Initialize macroscopic variables: density(height) and velocity field
        self.h = np.ones(self.Nx)  # height field
        self.u = np.zeros(self.Nx)  # Velocity field

        # Initialize distribution functions (f_i), f has 3 directions (D1Q3)
        self.f = np.zeros((self.Nx, 3))  # Distribution functions for D1Q3
        self.feq = np.zeros_like(self.f)  # Equilibrium distribution functions

    def get_F(self):

        g=self.g
        c=self.c_s

        F1 = 1/(self.tau*self.kn) [
        [0, 1, 1 ],
        [0, (1 / (2 * c)) - 1, -(1 / (2 * c))],
        [0, 1 - (1 / (2 * c)), 1 / (2 * c)]
        ]

        F2 = 1/(self.tau*self.kn*2*c**2)[
        [-g, -g, -g, -g , -g - 4, -g + 4, -g, -g + 4, -g - 4],
        [g, g, g, g, g + 2, g - 2, g, g - 2, g + 2],
        [g, g, g, g, g + 2, g - 2, g, g - 2, g + 2]
        ]

        F3 =[
        [0, 0, 0, 0 , 2, -2, 0, -2, 2],
        [0, 0, 0, 0, -1, 1, 0, 1, -1],
        [0, 0, 0, 0, -1, 1, 0, 1, -1]
        ]

        F3 =  1/(self.tau*self.kn*2*c**2)*np.kron(F3,F3,F3)

        return F1,F2,F3


    def get_C(self):

        F1,F2,F3 = self.get_F()

        A11=F1
        A12=F2
        A13=F3

        I1 = np.eye(F1.shape[0])
        I2= np.eye(F2.shape[0])

        A22 = np.kron(I1,F1) + np.kron(F1,I1)
        A23 = np.kron(I2,F2) + np.kron(F2,I2)
        A33 = np.kron(I1,I1,F1) + np.kron(F1,I1,I1) + np.kron(I1,F1,I1)

        zero1 = np.zeros((A22.shape[0], A11.shape[1]))  # Zero block for row 2
        zero2 = np.zeros((A33.shape[0], A11.shape[1] + A22.shape[1]))  # Zero block for row 3
            
        # Stack rows together
        row1 = np.hstack([A11, A12, A13])  # First row
        row2 = np.hstack([zero1, A22, A23])  # Second row
        row3 = np.hstack([zero2, A33])  # Third row
            
        # Stack all rows vertically
        block_matrix = np.vstack([row1, row2, row3])



